In [1]:
import torch
import torch.nn as nn
import torchvision.models as models
import torch.nn.functional as F
import numpy as np

## 依照指示取出模型特定層的資訊

In [2]:
# Define model
class TheModelClass(nn.Module):
    def __init__(self):
        super(TheModelClass, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16* 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Initialize model
model = TheModelClass()


### 取出 self.pool層兩次的輸出，包含：
* x = self.pool(F.relu(self.conv1(x)))
* x = self.pool(F.relu(self.conv2(x)))

In [3]:
outputs= []
def layer1_hook(module, input_, output):
    outputs.append(output)
    
model.pool.register_forward_hook(layer1_hook)

In [4]:
input_ = torch.randn(1, 3, 32, 32)
output = model(input_)

In [5]:
outputs

[tensor([[[[0.4592, 0.2721, 1.1302,  ..., 0.3732, 1.0189, 1.0619],
           [0.3363, 1.1297, 0.5216,  ..., 0.4868, 0.4308, 0.6677],
           [0.6244, 0.3281, 1.3942,  ..., 0.7258, 0.4583, 1.0050],
           ...,
           [1.0989, 0.3520, 0.9501,  ..., 0.7051, 0.5820, 0.8419],
           [0.7901, 0.4232, 0.5084,  ..., 0.6386, 0.8296, 0.9083],
           [0.0000, 1.3246, 1.0841,  ..., 0.0000, 0.5367, 1.0866]],
 
          [[1.2732, 0.3723, 0.4600,  ..., 0.6465, 0.6814, 0.3940],
           [0.8055, 0.7718, 0.2512,  ..., 0.7815, 0.9823, 0.5999],
           [0.4694, 0.9881, 0.4773,  ..., 0.5717, 0.2373, 0.4474],
           ...,
           [1.0704, 0.8337, 0.0000,  ..., 0.2003, 0.3623, 0.6199],
           [0.5859, 0.9357, 0.2539,  ..., 0.6086, 1.3262, 0.6559],
           [0.5362, 0.2661, 1.0625,  ..., 0.1566, 0.0745, 0.6069]],
 
          [[0.4823, 0.6875, 1.0330,  ..., 0.7949, 0.4428, 0.0589],
           [0.7718, 0.0252, 0.0000,  ..., 0.6249, 0.6189, 0.3776],
           [0.5263, 0.31

In [6]:
print(outputs[0].shape)
print(outputs[1].shape)

torch.Size([1, 6, 14, 14])
torch.Size([1, 16, 5, 5])


## 加入自定義 initialization fuction

#### 對所有Conv2D層使用自定義initialization function
* weight : nn.init.kaiming_normal_
* bias : 全部輸入1

In [7]:
from torch.nn import init

def weights_init(m):
    #classname = m.__class__.__name__
    if isinstance(m, nn.Conv2d):
        #torch.nn.init.xavier_uniform_(m.weight)
        nn.init.kaiming_normal_(m.weight.data,
                                    a=0,
                                    mode='fan_out',
                                    nonlinearity='relu')
        if m.bias is not None:
            m.bias.data.fill_(1)
            
model.apply(weights_init)

TheModelClass(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)

#### 查看 conv層的bias是否皆為1

In [8]:
for name, parameters in model.named_parameters():
    if ('conv' in name) and ('bias' in name):
        print(name, parameters)
        print('\n')

conv1.bias Parameter containing:
tensor([1., 1., 1., 1., 1., 1.], requires_grad=True)


conv2.bias Parameter containing:
tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       requires_grad=True)


